<a href="https://colab.research.google.com/github/Recommendation-System-Project/Recommendation-system/blob/main/DR_Final_Expt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib import style
import time

In [ ]:
# def get_data(sample_num=2000):
#     observed_r = np.random.uniform(low=0, high=5, size=(sample_num,)) # observed_r
#     predicted_r = np.random.uniform(low=0, high=5, size=(sample_num,)) # predicted rating
# #     imputed_e = np.power((predicted_r - 0.5), 2)*0.3 ## e_hat_ui = 0.3*(r_ui - 0.5)^2
    
#     observed_e = predicted_r - observed_r # e_ui
#     propensities = np.random.uniform(low=0, high=1, size=(sample_num,)) # propensities for observed X_ui
   
#     return observed_r, predicted_r, observed_e, propensities

## Import preproceed data

待做事项：
将predicted_r 替换成FM的预测值

In [3]:
def get_data():
    observed_r = pd.read_csv("preproceed_data/rating_pairs.csv").drop("user_id", axis = 1) # observed ratings
    observed_r.columns = range(1,len(observed_r.columns)+1)
    
    predicted_r = pd.DataFrame(np.random.randint(1,5,size=observed_r.shape)) # predicted rating
    predicted_r.columns = range(1,len(predicted_r.columns)+1)
    
    observed_e = predicted_r - observed_r # e_ui
    propensities = pd.read_csv("preproceed_data/propensities.csv", header = None) # propensities for all user item pairs
    propensities.columns = range(1,len(propensities.columns)+1)
    
    O_mask = pd.read_csv("preproceed_data/O_mask.csv").drop("user_id", axis = 1)
    O_mask.columns = range(1,len(O_mask.columns)+1)

    item_features = np.array(pd.read_csv("preproceed_data/item_features.csv", header = None))
    user_features = np.array(pd.read_csv("preproceed_data/user_features.csv", header = None))
   
    return observed_r, predicted_r, observed_e, propensities, O_mask, item_features, user_features

## Stochastic gradient descent

In [96]:
def SGD(predicted_r,
        observed_e,
        propensities,
        step_size=2,
        max_iter_count=2000):

    m = predicted_r.shape[0]
    var = 1
    loss = 1
    iter_count = 0
    iter_list = []
    loss_list = []
    theta1 = []
    theta2 = []
    theta = [2.2, 0.58]

    
    while loss > 0.01 and iter_count < max_iter_count:
        loss = 0
        theta1.append(theta[0])
        theta2.append(theta[1])
        rand1 = np.random.randint(0, m, 1)

        gradient_w = 2 * predicted_r[rand1] * (
            2 * theta[1] - predicted_r[rand1]) * (
                (2 * predicted_r[rand1] * theta[1] - predicted_r[rand1]**2) *
                theta[0] + observed_e[rand1] - theta[1]**2) / propensities[rand1]

        gradient_y = 4 * (theta[1] - theta[0] * predicted_r[rand1]) * (
            theta[1]**2 - 2 * predicted_r[rand1] * theta[0] * theta[1] -
            observed_e[rand1] + theta[0] * predicted_r[rand1]**2) / propensities[rand1]
        
        gradient = [gradient_w, gradient_y]
        
        for i in range(len(theta)):
            theta[i] = theta[i] - step_size * gradient[i]
        h = np.power((predicted_r - theta[1]), 2) * theta[0]
        
        for i in range(m):
            every_loss = np.power((h[i] - observed_e[i]), 2) / propensities[i]
            loss = loss + every_loss

        print("iter_count: ", iter_count, "the loss: ",loss)
        print("theta:", theta, "gradient: ", gradient)

        iter_list.append(iter_count)
        loss_list.append(loss)

        iter_count += 1

    plt.plot(iter_list, loss_list)
    plt.xlabel("iter")
    plt.ylabel("loss")
    plt.show()
    return theta1, theta2, theta, loss_list

## Process data the imported data 

In [4]:
observed_r, predicted_r, observed_e, propensities, O_mask, item_features, user_features = get_data()
    
# condense data into 1d array with only observed O(u,i)
observed_r_1d = np.array(observed_r*O_mask).flatten()
observed_r_1d = observed_r_1d[~np.isnan(observed_r_1d)]

predicted_r_1d = np.array(predicted_r*O_mask).flatten()
predicted_r_1d = predicted_r_1d[predicted_r_1d != 0]
    
observed_e_1d = np.array(observed_e).flatten()
observed_e_1d = observed_e_1d[~np.isnan(observed_e_1d)]
    
propensities_1d = np.array(propensities*O_mask).flatten()
propensities_1d = propensities_1d[propensities_1d != 0]

# creating features for all user-item pairs
observation_matrix = []
for u in range(user_features.shape[0]):
  for i in range(item_features.shape[0]):
    user_f = user_features[u]
    item_f = item_features[i]
    observed = np.concatenate((user_f, item_f))
    observation_matrix.append(observed)
observation_matrix = np.array(observation_matrix)

# select features for only observed user-item pairs
O_mask_1d = np.array(O_mask).flatten()
observed_features = []
for i in range(len(O_mask_1d)):
  if O_mask_1d[i] == 1:
    observed_features.append(observation_matrix[i])
observed_features = np.array(observed_features)

In [5]:
print(observed_r.shape)
observed_r.head()

(943, 1682)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,1.0,3.0,3.0,5.0,4.0,2.0,1.0,2.0,2.0,3.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print(observed_e.shape)
observed_e.head()

(943, 1682)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,-3.0,-1.0,-2.0,0.0,-1.0,-1.0,-2.0,1.0,-2.0,0.0,2.0,-1.0,-2.0,-3.0,-3.0,-1.0,-1.0,-2.0,-4.0,0.0,0.0,-2.0,0.0,-1.0,0.0,0.0,-1.0,-1.0,1.0,1.0,0.0,-3.0,-2.0,1.0,0.0,2.0,2.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,-2.0,-1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,NaN,NaN,-2.0,0.0,NaN,NaN,NaN,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
print(propensities.shape)
propensities.head()

(943, 1682)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,0.575451,0.222978,0.209602,0.241303,0.241137,0.351731,0.352477,0.358976,0.415236,0.387641,0.333544,0.316788,0.286183,0.336193,0.313439,0.254208,0.212679,0.259290,0.242214,0.239103,0.181182,0.202289,0.193698,0.163521,0.170176,0.171173,0.199320,0.144158,0.147585,0.184181,0.160672,0.183913,0.165150,0.137228,0.152506,0.181107,0.192479,0.180113,0.146492,0.191144,...,0.009034,0.009006,0.008991,0.008978,0.008962,0.008926,0.008894,0.008884,0.008866,0.008840,0.008820,0.008793,0.008782,0.008761,0.008745,0.008715,0.008689,0.008674,0.008653,0.008641,0.008625,0.008586,0.008571,0.008550,0.008530,0.008504,0.008505,0.008466,0.008448,0.008428,0.008409,0.008388,0.008368,0.008348,0.008328,0.008308,0.008290,0.008277,0.008243,0.008229
1,0.380192,0.114939,0.107150,0.125823,0.125723,0.197135,0.197653,0.202189,0.243199,0.222683,0.184665,0.173441,0.153572,0.186462,0.171228,0.133640,0.108931,0.136754,0.126370,0.124503,0.091022,0.102946,0.098056,0.081277,0.084925,0.085474,0.101250,0.070828,0.072660,0.092697,0.079725,0.092547,0.082167,0.067147,0.075304,0.090980,0.097366,0.090426,0.072075,0.096612,...,0.004109,0.004096,0.004089,0.004083,0.004076,0.004059,0.004045,0.004040,0.004032,0.004020,0.004011,0.003999,0.003993,0.003984,0.003976,0.003963,0.003951,0.003944,0.003935,0.003929,0.003922,0.003904,0.003897,0.003888,0.003878,0.003866,0.003867,0.003849,0.003841,0.003832,0.003823,0.003813,0.003804,0.003795,0.003786,0.003777,0.003769,0.003763,0.003747,0.003741
2,0.488474,0.168173,0.157418,0.183055,0.182919,0.276543,0.277197,0.282915,0.333455,0.308428,0.260680,0.246232,0.220246,0.262979,0.243364,0.193640,0.159885,0.197832,0.183799,0.181259,0.134866,0.151577,0.144749,0.121053,0.126240,0.127019,0.149213,0.106081,0.108718,0.137227,0.118839,0.137016,0.122321,0.100766,0.112514,0.134807,0.143783,0.134026,0.107876,0.142726,...,0.006382,0.006362,0.006351,0.006342,0.006331,0.006305,0.006283,0.006275,0.006262,0.006244,0.006230,0.006211,0.006203,0.006188,0.006177,0.006156,0.006137,0.006127,0.006112,0.006104,0.006092,0.006064,0.006053,0.006039,0.006024,0.006006,0.006007,0.005980,0.005967,0.005953,0.005939,0.005924,0.005910,0.005896,0.005882,0.005868,0.005855,0.005846,0.005822,0.005811
3,0.575451,0.222978,0.209602,0.241303,0.241137,0.351731,0.352477,0.358976,0.415236,0.387641,0.333544,0.316788,0.286183,0.336193,0.313439,0.254208,0.212679,0.259290,0.242214,0.239103,0.181182,0.202289,0.193698,0.163521,0.170176,0.171173,0.199320,0.144158,0.147585,0.184181,0.160672,0.183913,0.165150,0.137228,0.152506,0.181107,0.192479,0.180113,0.146492,0.191144,...,0.009034,0.009006,0.008991,0.008978,0.008962,0.008926,0.008894,0.008884,0.008866,0.008840,0.008820,0.008793,0.008782,0.008761,0.008745,0.008715,0.008689,0.008674,0.008653,0.008641,0.008625,0.008586,0.008571,0.008550,0.008530,0.008504,0.008505,0.008466,0.008448,0.008428,0.008409,0.008388,0.008368,0.008348,0.008328,0.008308,0.008290,0.008277,0.008243,0.008229
4,0.423142,0.134423,0.125501,0.146845,0.146732,0.226979,0.227552,0.232576,0.277605,0.255165,0.213121,0.200594,0.178285,0.215123,0.198118,0.155735,0.127543,0.159269,0.147469,0.145341,0.106941,0.120674,0.115050,0.095671,0.099895,0.100530,0.118724,0.083540,0.085671,0.108874,0.093872,0.108701,0.096703,0.079255,0.088742,0.106893,0.114255,0.106253,0.084990,0.113386,...,0.004909,0.004894,0.004886,0.004879,0.004870,0.004851,0.004833,0.004827,0.004817,0.004803,0.004793,0.004778,0.004772,0.004760,0.004751,0.004735,0.004721,0.004713,0.004702,0.004695,0.004686,0.004665,0.004656,0.004645,0.004634,0.004620,0.004621,0.004600,0.004590,0.004579,0.004568,0.004557,0.004546,0.004535,0.004524,0.004513,0.004503,0.004496,0.004478,0.004470


In [8]:
print(O_mask.shape)
O_mask.head()

(943, 1682)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
print(O_mask.shape)
O_mask.head()

(943, 1682)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
print(observed_r_1d.shape)
observed_r_1d

(100000,)


array([5., 3., 4., ..., 3., 3., 3.])

In [12]:
print(O_mask.shape)
O_mask.head()

(943, 1682)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
print(O_mask_1d.shape)
O_mask_1d

(1586126,)


array([1, 1, 1, ..., 0, 0, 0])

In [16]:
print(observed_e_1d.shape)
observed_e_1d

(100000,)


array([-3., -1., -2., ...,  1., -1.,  0.])

In [18]:
print(propensities_1d.shape)
propensities_1d

(100000,)


array([0.57545082, 0.22297829, 0.20960195, ..., 0.01936397, 0.01759446,
       0.01381045])

In [20]:
print(observation_matrix.shape)
observation_matrix

(1586126, 20)


array([[2.40000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 4.37761021e+00, 0.00000000e+00],
       [2.40000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 3.64613199e+00, 0.00000000e+00],
       [2.40000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.80150759e+00, 0.00000000e+00],
       ...,
       [2.20000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        9.44601929e+02, 0.00000000e+00, 1.16251782e+03],
       [2.20000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        9.45498108e+02, 0.00000000e+00, 1.16361829e+03],
       [2.20000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        9.45951782e+02, 0.00000000e+00, 1.16417822e+03]])

In [22]:
print(observed_features.shape)
observed_features

(100000, 20)


array([[ 24.        ,   1.        ,   0.        , ...,   0.        ,
          4.37761021,   0.        ],
       [ 24.        ,   1.        ,   0.        , ...,   0.        ,
          3.64613199,   0.        ],
       [ 24.        ,   1.        ,   0.        , ...,   0.        ,
          1.80150759,   0.        ],
       ...,
       [ 22.        ,   1.        ,   0.        , ..., 667.7911377 ,
          0.        , 821.8371582 ],
       [ 22.        ,   1.        ,   0.        , ..., 690.5925293 ,
          0.        , 849.90124512],
       [ 22.        ,   1.        ,   0.        , ..., 747.95306396,
          0.        , 920.49755859]])

## Train imputation error model
注意，训练需要花费500-700s不等

In [ ]:
start = time.time()
theta1,theta2,theta,loss_list = SGD(predicted_r_1d, observed_e_1d, propensities_1d, step_size = 0.0001)
end = time.time()
    
print("The time of execution of above program is :", end-start)
print("theta for imputation error: ", theta)

## Factor Machine using Tensorflow

In [23]:
from itertools import count # 迭代器
from collections import defaultdict # 使用dict时，如果引用的Key不存在，就会抛出KeyError。如果希望key不存在时，返回一个默认值，就可以用defaultdict
from scipy.sparse import csr # csr_matrix，全名为Compressed Sparse Row，是按行对矩阵进行压缩的。CSR需要三类数据：数值，列号，以及行偏移量。CSR是一种编码的方式，其中，数值与列号的含义，与coo里是一致的。行偏移表示某一行的第一个元素在values里面的起始偏移位置。 
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tqdm import tqdm_notebook as tqdm # 可以显示循环的进度条的库

Instructions for updating:
non-resource variables are not supported in the long term


待完成：
注意！此处x_train, y_train需要替换成做所有的useritem pair features

In [29]:
x_train = observation_matrix.astype(np.float32)
y_train = np.array(observed_r).flatten().astype(np.float32)

In [30]:
theta = np.array(theta).astype(np.float32)

NameError: ignored

In [136]:
n,p = x_train.shape

k = 10

x = tf.placeholder('float',[None,p])

y = tf.placeholder('float',[None,1])

O_ui = tf.Variable(O_mask_1d.astype(np.float32))
P_ui = tf.Variable(np.array(propensities).flatten().astype(np.float32))

w0 = tf.Variable(tf.zeros([1]))
w = tf.Variable(tf.zeros([p]))

v = tf.Variable(tf.random_normal([k,p],mean=0,stddev=0.01))

#y_hat = tf.Variable(tf.zeros([n,1]))

linear_terms = tf.add(w0,tf.reduce_sum(tf.multiply(w,x),1,keep_dims=True)) # n * 1
pair_interactions = 0.5 * tf.reduce_sum(
    tf.subtract(
        tf.pow(
            tf.matmul(x,tf.transpose(v)),2),
        tf.matmul(tf.pow(x,2),tf.transpose(tf.pow(v,2)))
    ),axis = 1 , keep_dims=True)

y_hat = tf.add(linear_terms,pair_interactions)

In [137]:
lambda_w = tf.constant(0.001,name='lambda_w')
lambda_v = tf.constant(0.001,name='lambda_v')

l2_norm = tf.reduce_sum(
    tf.add(
        tf.multiply(lambda_w,tf.pow(w,2)),
        tf.multiply(lambda_v,tf.pow(v,2))
    )
)

imputed_error = tf.reduce_sum(tf.multiply(theta[0],tf.square(y_hat - theta[1])))
inner = tf.subtract(tf.subtract(y, y_hat), imputed_error)
error = tf.divide(tf.add(imputed_error, tf.multiply(O_ui, inner)), P_ui)
# error = tf.reduce_mean(tf.square(y-y_hat))
loss = tf.add(error,l2_norm)

train_op = tf.train.GradientDescentOptimizer(learning_rate=0.000001).minimize(loss)

In [ ]:
# Launch the graph
epochs = 100
init = tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)

  for epoch in tqdm(range(epochs), unit='epoch'):
    sess.run(train_op, feed_dict={x: x_train.reshape(-1, p), y: y_train.reshape(-1, 1)})
    print("loss: ", sess.run(loss, feed_dict={x: x_train.reshape(-1, p), y: y_train.reshape(-1, 1)}))
    print("y_hat: ", sess.run(y_hat,feed_dict={x: x_train.reshape(-1, p), y: y_train.reshape(-1, 1)}))
    print("V: ", sess.run(pair_interactions,feed_dict={x: x_train.reshape(-1, p), y: y_train.reshape(-1, 1)}))
    
  
  errors = sess.run(error,feed_dict={x: x_train.reshape(-1, p), y: y_train.reshape(-1, 1)})
  print("error: ", errors)
  RMSE = np.sqrt(np.array(errors).mean())
  print("RMSE: ", RMSE)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/100 [00:00<?, ?epoch/s]

In [75]:
y_hat

<tf.Tensor 'Add_9:0' shape=(?, 1) dtype=float32>